In [ ]:
!pip install transformers torch
!pip install datasets
!pip install evaluate
!pip install bert_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
import evaluate

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
prompt = "Words hold immense power. They can inspire, uplift, and transform lives. A well-crafted sentence has the ability to ignite imagination, stir emotions, and leave an indelible mark on the human psyche. It's a testament to the remarkable capacity of language to transcend barriers and forge connections."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids


In [ ]:
# Set top_k to a value like 50 as an example
outputs3 = model.generate(input_ids, do_sample=False, top_k=1000, max_length=50)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:427: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1000` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


In [ ]:
generated_text = tokenizer.batch_decode(outputs3, skip_special_tokens=True)
print(generated_text)


["Words hold immense power. They can inspire, uplift, and transform lives. A well-crafted sentence has the ability to ignite imagination, stir emotions, and leave an indelible mark on the human psyche. It's a testament"]


In [ ]:
encodings = tokenizer(generated_text, return_tensors="pt").input_ids


In [ ]:
device = "cpu"
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl

  0%|          | 0/1 [00:02<?, ?it/s]


tensor(1.3566)

In [ ]:
prompt = "The natural world is a tapestry of wonders. From the majestic peaks of snow-capped mountains to the intricate dance of life in the depths of the oceans, nature's artistry is a constant source of awe and reverence. It reminds us of our humble place in the grand scheme of existence and our responsibility to protect its fragile beauty."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs1 = model.generate(input_ids, do_sample=True, top_k=50, max_length=30)
generated_text = tokenizer.batch_decode(outputs1, skip_special_tokens=True)
print(generated_text)
encodings = tokenizer(generated_text, return_tensors="pt").input_ids

device = "cpu"
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl


["Nature's artistry is a constant source of awe and reverence. It reminds us of our humble place in the grand scheme of existence."]


  0%|          | 0/1 [00:00<?, ?it/s]


tensor(1.2304)

In [ ]:
prompt = "Creativity is the lifeblood of progress. It fuels innovation, drives artistic expression, and propels us forward. In a world that often values conformity, embracing our creative spirit allows us to challenge the status quo, question the established norms, and pave the way for new possibilities."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs2 = model.generate(input_ids, do_sample=True, top_k=50, max_length=30)
generated_text = tokenizer.batch_decode(outputs2, skip_special_tokens=True)
print(generated_text)
encodings = tokenizer(generated_text, return_tensors="pt").input_ids

device = "cpu"
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl


['Creativity is the lifeblood of progress. It fuels innovation, drives artistic expression, and propels us forward. In a world that']


  0%|          | 0/1 [00:00<?, ?it/s]


tensor(1.4054)

In [ ]:
prompt = "Resilience is the hallmark of the human spirit. It is the ability to rise above adversity, to adapt and evolve in the face of challenges. Like a sturdy oak that withstands the fiercest storms, resilience empowers us to bend but never break, emerging stronger and wiser from life's trials."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs2 = model.generate(input_ids, do_sample=True, top_k=50, max_length=30)
generated_text = tokenizer.batch_decode(outputs2, skip_special_tokens=True)
print(generated_text)
encodings = tokenizer(generated_text, return_tensors="pt").input_ids

device = "cpu"
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl


['Resilience is the hallmark of the human spirit. It is the ability to rise above adversity. Like a sturdy oak that withstands']


  0%|          | 0/1 [00:00<?, ?it/s]


tensor(1.5952)

In [ ]:
prompt = "Kindness is a universal language that transcends boundaries and cultures. A simple act of compassion, a gentle word of encouragement, or a genuine smile can have a profound impact, creating ripples of positivity that touch the lives of others. In a world that often appears harsh and unforgiving, kindness is a beacon of hope and humanity."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs2 = model.generate(input_ids, do_sample=True, top_k=50, max_length=30)
generated_text = tokenizer.batch_decode(outputs2, skip_special_tokens=True)
print(generated_text)
encodings = tokenizer(generated_text, return_tensors="pt").input_ids

device = "cpu"
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
ppl


['Kindness is a universal language that transcends boundaries and cultures. A simple act of compassion, a gentle word of encouragement, or a']


  0%|          | 0/1 [00:00<?, ?it/s]


tensor(1.4771)

# **Task 2**

In [ ]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '2.0.0')
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
test_dataset = dataset["test"]

sentence_list = []
reference_list = []

# Iterate through elements in the test dataset, stopping at 50 items
i = 0
for item in test_dataset:
    if i == 50:
        break  # Stop after processing 50 items
    sentence_list.append(item["article"])
    reference_list.append(item["highlights"])
    i+=1


In [ ]:
prediction_list = []
for i in range(len(sentence_list)):
  input_ids = tokenizer(sentence_list[i], return_tensors="pt", max_length=1024, truncation=True).input_ids
  output = model.generate(input_ids, do_sample=True, top_k=50, max_length=30)
  print(tokenizer.batch_decode(output, skip_special_tokens=True)[0])
  prediction_list.append(tokenizer.batch_decode(output, skip_special_tokens=True)[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1202: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (30). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories.
Theia was hit by a car and buried in a field, then found four days later. She suffered a dislocated jaw, leg
Mohammad Javad Zarif is the Iranian foreign minister. He has been John Kerry's opposite number in securing a breakthrough in nuclear discussions
The five were exposed to Ebola in Sierra Leone in March. None developed the deadly virus. They are clinicians for Partners in Health, a
The student is no longer on campus and will face student conduct review, the university says. Officials are still trying to determine if other people
Trey Moses asked Ellie Meredith, a freshman with Down syndrome, to be his prom date. Photos of the proposal have gone viral.
Amnesty International's annual report catalogs the use of state-sanctioned killing as a punitive measure. At least 607 people
Andrew Getty, grandson of oil tycoo

In [ ]:
print("Input")
for i in range(len(sentence_list)):
      print(sentence_list[i][:1024])


Input
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremo

In [ ]:
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=prediction_list, references=reference_list, use_aggregator=False)
print("Rouge Results: ", results)

Rouge Results:  {'rouge1': [0.44827586206896547, 0.39999999999999997, 0.48275862068965525, 0.4242424242424242, 0.5074626865671642, 0.43478260869565216, 0.18918918918918917, 0.28125000000000006, 0.44897959183673475, 0.4878048780487805, 0.4150943396226416, 0.2553191489361702, 0.24489795918367346, 0.17391304347826086, 0.058823529411764705, 0.12307692307692307, 0.4912280701754386, 0.08823529411764705, 0.43478260869565216, 0.2692307692307693, 0.2105263157894737, 0.25, 0.3692307692307692, 0.631578947368421, 0.5660377358490566, 0.19718309859154928, 0.3111111111111111, 0.19047619047619047, 0.16666666666666666, 0.4782608695652174, 0.30379746835443033, 0.18461538461538457, 0.35000000000000003, 0.20833333333333331, 0.3508771929824561, 0.25000000000000006, 0.3508771929824562, 0.34920634920634924, 0.27586206896551724, 0.17777777777777778, 0.6222222222222222, 0.23999999999999996, 0.6956521739130435, 0.5, 0.4186046511627907, 0.3111111111111111, 0.8799999999999999, 0.32, 0.23076923076923078, 0.3720930

In [ ]:
for item in results["rouge1"]:
  print(item)

0.44827586206896547
0.39999999999999997
0.48275862068965525
0.4242424242424242
0.5074626865671642
0.43478260869565216
0.18918918918918917
0.28125000000000006
0.44897959183673475
0.4878048780487805
0.4150943396226416
0.2553191489361702
0.24489795918367346
0.17391304347826086
0.058823529411764705
0.12307692307692307
0.4912280701754386
0.08823529411764705
0.43478260869565216
0.2692307692307693
0.2105263157894737
0.25
0.3692307692307692
0.631578947368421
0.5660377358490566
0.19718309859154928
0.3111111111111111
0.19047619047619047
0.16666666666666666
0.4782608695652174
0.30379746835443033
0.18461538461538457
0.35000000000000003
0.20833333333333331
0.3508771929824561
0.25000000000000006
0.3508771929824562
0.34920634920634924
0.27586206896551724
0.17777777777777778
0.6222222222222222
0.23999999999999996
0.6956521739130435
0.5
0.4186046511627907
0.3111111111111111
0.8799999999999999
0.32
0.23076923076923078
0.37209302325581395


In [ ]:
results = bert.compute(predictions=prediction_list, references=reference_list, lang="en")
for item in results["precision"]:
  print(item)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.9076894521713257
0.8897876143455505
0.9066729545593262
0.8987829685211182
0.9193089008331299
0.9174870252609253
0.8718336820602417
0.8702742457389832
0.888621985912323
0.9235469102859497
0.8690422177314758
0.8847770690917969
0.86968994140625
0.8586580753326416
0.7976261973381042
0.8375508189201355
0.9152008891105652
0.858532190322876
0.9257023334503174
0.8703681826591492
0.8637768030166626
0.8769705295562744
0.8297592997550964
0.95368492603302
0.904296875
0.8581005930900574
0.8746227622032166
0.8383881449699402
0.8611944317817688
0.8951525688171387
0.9106804132461548
0.8655833005905151
0.89287269115448
0.8380581736564636
0.8978191614151001
0.8830930590629578
0.9008278250694275
0.8926330208778381
0.8751252293586731
0.8368083238601685
0.9008232951164246
0.8309321403503418
0.9355967044830322
0.950108528137207
0.8792961835861206
0.8929635286331177
0.9751830697059631
0.8963255286216736
0.8681859970092773
0.8899294137954712


In [ ]:
from nltk import agreement
rater1 = [5, 5, 5, 4, 5, 5, 4, 5, 4, 5, 5, 3, 4, 4, 4, 5, 4, 4, 4, 4]
rater2 = [4, 3, 4, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5]
rater3 = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
rater4 = [4, 4, 5, 3, 4, 5, 3, 4, 2, 3, 4, 3, 4, 3, 2, 5, 3, 4, 5, 5]

taskdata=[[0,str(i),str(rater1[i])] for i in range(0,len(rater1))]+[[1,str(i),str(rater2[i])]
for i in range(0,len(rater2))]+[[2,str(i),str(rater3[i])] for i in range(0,len(rater3))]+[[1,str(i),str(rater4[i])]
for i in range(0,len(rater4))]

ratingtask = agreement.AnnotationTask(data=taskdata)
print("alpha " +str(ratingtask.alpha()))


alpha -0.011129660545353248
